In [ ]:
#Menampilkan data
import pandas as pd

df = pd.read_excel('credit.xlsx')
df

,Unnamed: 0,Catatan_kredit,Utang,Jaminan,Pendapatan,Risiko
0,1,buruk,rendah,tidak_ada,0-15,tinggi
1,2,tidak_diketahui,tinggi,tidak_ada,15-35,tinggi
2,3,tidak_diketahui,rendah,tidak_ada,15-35,rendah
3,4,buruk,rendah,tidak_ada,0-15,tinggi
4,5,tidak_diketahui,rendah,ada,>35,rendah
...,...,...,...,...,...,...
115,116,baik,tinggi,ada,15-35,rendah
116,117,baik,tinggi,tidak_ada,15-35,rendah
117,118,baik,tinggi,ada,15-35,rendah
118,119,tidak_diketahui,rendah,tidak_ada,15-35,tinggi


In [ ]:
#Data preparation (cek tipe data dan missing values)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      120 non-null    int64 
 1   Catatan_kredit  120 non-null    object
 2   Utang           120 non-null    object
 3   Jaminan         120 non-null    object
 4   Pendapatan      120 non-null    object
 5   Risiko          120 non-null    object
dtypes: int64(1), object(5)
memory usage: 5.8+ KB


In [ ]:
df.isna().sum()

Unnamed: 0        0
Catatan_kredit    0
Utang             0
Jaminan           0
Pendapatan        0
Risiko            0
dtype: int64

In [ ]:
#import label encoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
#konversi nilai variabel ke numerik
label_encoder = LabelEncoder()
categorical_columns = ['Catatan_kredit', 'Utang', 'Jaminan', 'Pendapatan']

for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

In [ ]:
df

,Unnamed: 0,Catatan_kredit,Utang,Jaminan,Pendapatan,Risiko
0,1,1,0,1,0,tinggi
1,2,2,1,1,1,tinggi
2,3,2,0,1,1,rendah
3,4,1,0,1,0,tinggi
4,5,2,0,0,2,rendah
...,...,...,...,...,...,...
115,116,0,1,0,1,rendah
116,117,0,1,1,1,rendah
117,118,0,1,0,1,rendah
118,119,2,0,1,1,tinggi


In [ ]:
#import library
from sklearn.model_selection import train_test_split

#deklarasi data X dan y
X = df.drop('Risiko', axis=1)
y = df['Risiko']

#splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#y_train must be encoded in a newer update XGBoost model before training it
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
#Membangun Model XGBoost
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

#hasil nilai prediksi
y_pred = model.predict(X_test)
print("Predicted Values:")
print(y_pred)
print("Actual Values:")
print(y_test.values)

Predicted Values:
[1 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0]
Actual Values:
['tinggi' 'rendah' 'rendah' 'tinggi' 'rendah' 'rendah' 'rendah' 'tinggi'
 'tinggi' 'rendah' 'tinggi' 'rendah' 'tinggi' 'rendah' 'rendah' 'rendah'
 'tinggi' 'tinggi' 'rendah' 'tinggi' 'tinggi' 'tinggi' 'tinggi' 'rendah']


In [ ]:
#inverse transform the predicted y values
y_pred = le.inverse_transform(y_pred)

In [ ]:
#Evaluasi model
from sklearn.metrics import confusion_matrix, classification_report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[ 8  4]
 [ 2 10]]

Classification Report:
              precision    recall  f1-score   support

      rendah       0.80      0.67      0.73        12
      tinggi       0.71      0.83      0.77        12

    accuracy                           0.75        24
   macro avg       0.76      0.75      0.75        24
weighted avg       0.76      0.75      0.75        24

